# Create an Azure AI Foundry Agent with Semantic Kernel

Follow these steps to create and interact with an Azure AI Foundry Agent using the Semantic Kernel SDK. 

You will build a bike inventory assistant for Contoso Bike Store.


**Step 1**: Install NuGet packages

Install the following NuGet packages to enable Azure AI Foundry and Semantic Kernel features:

In [12]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file so that they can be accessed by the application.

In [13]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project Client

Create a `PersistentAgentsClient` using your Azure AI Foundry project endpoint.

Note that **DefaultAzureCredential** is used for authentication. Run `az login --use-device-code` if you haven't already.

In [14]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;

var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

# Bike Inventory Agent

Build an agent that answers questions about bikes at Contoso Bike Store.

**Step 1:** Create the plugin to simulate the bike inventory.

In [15]:
using System.Collections.Generic;
using Microsoft.SemanticKernel;

public class BikeInventoryPlugin
{
    private readonly Dictionary<string, string> _inventory = new()
    {
        { "Contoso Explorer", "A rugged mountain bike suitable for off-road and trail riding." },
        { "Contoso CityLite", "A lightweight city bike perfect for commuting and urban rides." },
        { "Contoso TrailBlazer", "A versatile hybrid bike for both city and light trail use." },
        { "Contoso Speedster", "A fast road bike designed for speed and long-distance rides." }
    };

    [KernelFunction]
    public string CheckBikeAvailability(string model)
    {
        if (_inventory.ContainsKey(model))
            return $"Yes, the {model} is available. {model} details: {_inventory[model]}";
        return $"Sorry, the {model} is currently out of stock.";
    }

    [KernelFunction]
    public string ListAvailableBikes()
    {
        return "Available bikes: " + string.Join(", ", _inventory.Keys);
    }
}

**Step 2**: Create the bike inventory agent definition.

In this step, define the attributes for the agent, including its name, instructions, tools (if any), and the model to use.

In [ ]:
var modelId = Environment.GetEnvironmentVariable("TEXT_MODEL_NAME");
var agentName = "ContosoInventoryAgent_Basic";
var agentInstructions = @"
You are a helpful assistant for Contoso Bike Store. Use the BikeInventoryPlugin to answer questions about bike availability and features.
If the user asks for a bike that is not in stock, inform them politely.
If the user asks for a list of available bikes, provide the list.
If the user asks for other information, politely decline and suggest they ask about bikes in the store.
If the user asks for bike features, provide a short description if available.
";

var agentDefinition = await client.Administration.CreateAgentAsync(
    modelId,
    name: agentName,
    instructions: agentInstructions,
    tools: [],
    toolResources: null);
    

**Step 3**: Create the agent.

Use the definition from the previous step to create the agent.

In [17]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var agent = new AzureAIAgent(agentDefinition, client);

agent.Kernel.ImportPluginFromObject(new BikeInventoryPlugin(), "bikeInventory");

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4**: Create a helper function to call the agent and handle the response.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
{
    ChatMessageContent message = new(AuthorRole.User, input);
    await foreach (ChatMessageContent response in agent.InvokeAsync(message, agentThread))
    {
        Console.WriteLine($"{response.Role}: {response.Content}");
    }
};

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

assistant: Yes, the Contoso Speedster is available! It is a fast road bike designed for speed and long-distance rides. Let me know if you'd like to learn more or check other bikes.


**Step 5**: Now let us try invoking the agent with different queries.

#### Query 1: Check Bike Availability

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var thread = new AzureAIAgentThread(client);
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

await invokeAgentAsync(thread, "Is the Contoso Speedster available?");

assistant: Yes, the Contoso Speedster is available! It’s a fast road bike designed for speed and long-distance rides. Let me know if you have more queries.


#### Query 2: List All Available Bikes

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var thread = new AzureAIAgentThread(client);
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

await invokeAgentAsync(thread,"Can you list all available bikes at the Seatle store?");

assistant: I'm sorry, I can only provide information about the bikes available in the Contoso Bike Store but not specific locations like the Seattle store. Feel free to ask about bike availability or features!
